In [1]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict, Parse
from datetime import datetime
import json

In [5]:
def getDataset(): 
    __file__ = Path().cwd()
    
    gtfsRecords = os.path.join(__file__, 'GtfsRRecords.zip')

    with ZipFile(gtfsRecords, 'r') as zip:
        dirs = zip.namelist()

        for r in range(0, 1):
            realtime_data = zip.read(dirs[r])

            feed = gtfs_realtime_pb2.FeedMessage()
            Parse(realtime_data, feed)

            timestamp = datetime.fromtimestamp(feed.header.timestamp)
            print(timestamp)
            print(feed.entity)

    dataset = None
    return dataset

dataset = getDataset()
# dataset.head()

2021-01-11 06:44:34
[id: "1109408.2.10-769-e19-1.2334.I"
trip_update {
  trip {
    trip_id: "1109408.2.10-769-e19-1.2334.I"
    start_time: "04:53:00"
    start_date: "20210111"
    schedule_relationship: SCHEDULED
    route_id: "10-769-e19-1"
  }
  stop_time_update {
    stop_sequence: 1
    departure {
      delay: 0
    }
    stop_id: "8520B1358601"
    schedule_relationship: SCHEDULED
  }
  stop_time_update {
    stop_sequence: 2
    arrival {
      delay: 60
    }
    departure {
      delay: 120
    }
    stop_id: "8520B1015901"
    schedule_relationship: SCHEDULED
  }
  stop_time_update {
    stop_sequence: 4
    arrival {
      delay: 60
    }
    departure {
      delay: 60
    }
    stop_id: "8520B1016101"
    schedule_relationship: SCHEDULED
  }
  stop_time_update {
    stop_sequence: 12
    arrival {
      delay: -120
    }
    departure {
      delay: -120
    }
    stop_id: "8520B1016701"
    schedule_relationship: SCHEDULED
  }
  stop_time_update {
    stop_sequence: 13